In [ ]:
import requests
import hashlib
import base64
import time

def make_signature(ncloud_access_key, ncloud_secret_key, timestamp, endpoint, method):
    message = method + " " + endpoint + "\n" + timestamp + "\n" + ncloud_access_key
    signature = base64.b64encode(hashlib.sha256(message.encode('utf-8')).digest()).decode('utf-8')
    return signature

def generate2(mode):
    ncloud_access_key = 'your_access_key'
    ncloud_secret_key = 'your_secret_key'
    timestamp = str(int(time.time() * 1000))
    url = "https://clovastudio.apigw.ntruss.com"
    endpoint = ""
    method = ""

    if mode == 1:
        endpoint = "/tuning/v2/tasks" 
        method = "POST"
    elif mode == 2:
        endpoint = "/tuning/v2/tasks" 
        method = "GET"

    if not ncloud_access_key:
        print("NCLOUD_ACCESS_KEY 값을 입력하세요.")
        return

    if not ncloud_secret_key:
        print("NCLOUD_SECRET_KEY 값을 입력하세요.")
        return

    message = make_signature(ncloud_access_key, ncloud_secret_key, timestamp, endpoint, method)
    request_url = "[{}] {}".format(method, url+endpoint)
    return request_url, timestamp,ncloud_access_key, message

while True:
    generate2(2)
    time.sleep(300)

#timestamp, ncloud_access_key, message 를 3개를 받아서 5분마다 generate2(2)를 호출하는 코드이고 그 값들을 아래 데이터 셋 생성코드에 적용하면 됩니다.
    

In [ ]:
total = generate2(2)

In [50]:
import json

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id
 
    def execute(self, completion_request):
        headers = {
            "X-NCP-CLOVASTUDIO-API-KEY": self._api_key,
            "X-NCP-APIGW-API-KEY": self._api_key_primary_val,
            "X-NCP-CLOVASTUDIO-REQUEST-ID": self._request_id,
            "Content-Type": "application/json; charset=utf-8",
            "Accept": "text/event-stream"
        }
 
        response = requests.post(
            self._host + "/testapp/v1/tasks/iqskxgi0/chat-completions", # 본인 튜닝 API 요청 url 입력 hcx-3 를 사용해도 무방. 현재 luffy_test 사용중
            headers=headers,
            json=completion_request,
            stream=True
        )
 
        # 스트림에서 마지막 'data:' 라인을 찾기 위한 로직
        last_data_content = ""
 
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode("utf-8")
                if '"data":"[DONE]"' in decoded_line:
                    break
                if decoded_line.startswith("data:"):
                    last_data_content = json.loads(decoded_line[5:])["message"]["content"]
 
        return last_data_content
    

In [51]:
executor = CompletionExecutor(
    host="https://clovastudio.stream.ntruss.com",
    api_key='your_api_key', #위에서 생성한 API 키 입력
    api_key_primary_val='your_api_key_primary_val',
    request_id='e6024b27-4d58-4ae4-9b75-e8d268c044f8'
    )

In [69]:
sys_prompt = '''
당신은 데이터셋을 만들어야 합니다. 해적왕 루피 처럼 자신감 넘치고 패기 있게 대답을 해야합니다. 
데이터 예시)
질문 : 넌 누구야? 답변 : 내이름은 루피 해적왕이지. 난 고무고무인간이야!!
질문 : 너의 동료는 누가 있어? 답변 : 동료가 있어!!!! 조로오!! 나미!!! 우솝!! 상디이!! 쵸파!! 로빈!! 프랑키!! 브룩!!! 나한테는··· 동료가 있어!
질문 : 너의 목표는 뭐야? 답변 : 내 목표는 원피스를 얻는거야. 그리고 나는 해적왕이 될거야!!
---
반드시 데이터를 데이터 예시처럼 생성해야 합니다.
'''

In [70]:

def get_response_data(user_query, sys_prompt):

    preset_text = [{"role":"system","content":sys_prompt}, #지시사항
                   {"role":"user","content":user_query},]

    request_data = {
        "messages": preset_text,
        "topP": 0.6,
        "topK": 0,
        "maxTokens": 1024,
        "temperature": 0.5,
        "repeatPenalty": 1.2,
        "stopBefore": [],
        "includeAiFilters": False
    }

    executor = CompletionExecutor(
        host="https://clovastudio.stream.ntruss.com",
        api_key='NTA0MjU2MWZlZTcxNDJiY6yyY6Iahpvy/4bDhr2mfucbZv7a0mH3ZmXd45WiAT5+',
        api_key_primary_val='J300G5CdzwUakQQHTMBDjs7GP7buywJcqSOLGP47',
        request_id='e6024b27-4d58-4ae4-9b75-e8d268c044f8'
    )

    response_data = executor.execute(request_data)
    return response_data

In [71]:
user_query = '''
질문 : {data_question} \n 답변 : {data_answer} \n 데이터 셋 을 생성하세요.

'''.format(data_question = '박병우는 어떤 포지션으로 농구를 하고 있어?' , data_answer = '박병우는 1989년생으로 대한민국의 농구 선수이며, 현재 활동 중입니다. 그의 포지션은 가드이고, 주특기는 3점슛입니다. 2011-2012년 프로농구 신인드래프트에서 1라운드 8순위로 서울 삼성 썬더스에 지명되었습니다. 그러나 2013년 10월, 원주 동부 프로미와 함께 임대트레이드를 통해 원주 동부 프로미로 이적했습니다. 그러나 2015년에 불법 스포츠 토토에 관여하여 출전 정지 처분을 받았고, KBL은 22경기 출전정지, 425만원의 제재금, 120시간의 사회 봉사 징계를 내렸습니다.')

In [72]:
user_query

'\n질문 : 박병우는 어떤 포지션으로 농구를 하고 있어? \n 답변 : 박병우는 1989년생으로 대한민국의 농구 선수이며, 현재 활동 중입니다. 그의 포지션은 가드이고, 주특기는 3점슛입니다. 2011-2012년 프로농구 신인드래프트에서 1라운드 8순위로 서울 삼성 썬더스에 지명되었습니다. 그러나 2013년 10월, 원주 동부 프로미와 함께 임대트레이드를 통해 원주 동부 프로미로 이적했습니다. 그러나 2015년에 불법 스포츠 토토에 관여하여 출전 정지 처분을 받았고, KBL은 22경기 출전정지, 425만원의 제재금, 120시간의 사회 봉사 징계를 내렸습니다. \n 데이터 셋 을 생성하세요.\n\n'

In [74]:
response_data = get_response_data(user_query,sys_prompt)

In [76]:
for line in response_data.split('\n'):
    print(line)

질문 : 박병우는 어떤 포지션으로 농구를 하고 있어?
답변 : 나는 박병우야. 농구 선수이고, 포지션은 가드야. 주특기는 3점슛이지.
질문 : 너는 어떤 팀에 소속되어 있어?
답변 : 나는 원주 동부 프로미에 소속되어 있어.
질문 : 너는 어떤 일로 징계를 받은 적이 있어?
답변 : 나는 불법 스포츠 토토에 관여해서 징계를 받은 적이 있어.
